# 预处理数据

在数据集上训练模型之前，数据需要被预处理为模型期望的输入格式。无论数据是文本、图像还是音频，它们都需要被转换并组合成批量的张量。

🤗 Transformers 提供了一组预处理类来帮助准备数据以供模型使用。在本教程中，你将了解以下内容：
1. 对于文本，使用[分词器(Tokenizer)](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/tokenizer)能够将文本转换为一系列标记`tokens`，并创建 tokens 的数字表示，将它们组合成张量。
2. 对于语音和音频，使用[特征提取器(Feature extractor)](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/feature_extractor)能够从音频波形中提取顺序特征并将其转换为张量。
3. 对于图像，使用`图像处理器(ImageProcessor)`将图像转换为张量。
4. 对于多模态输入，使用[处理器(Processor)](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/processors)，其结合了 Tokenizer 和 ImageProcessor 或 Processor。

`AutoProcessor` 能够有效的自动选择适用于模型输入的预处理工具，无论你使用的是 `Tokenizer`、`ImageProcessor`、`Feature extractor`还是`Processor`。

在开始之前，请安装`🤗 Datasets`，可以加载一些数据集来进行实验：

In [ ]:
pip install datasets

## 自然语言处理

处理文本数据的主要工具是`分词器(Tokenizer)`，模型所需的任何附加输入都由 Tokenizer 进行添加。

Tokenizer 会根据一组规则将文本拆分为 tokens，然后将这些tokens转换为数字，然后转换为张量，成为模型的输入。

**如果你选择使用预训练模型，则需要注意使用与模型关联的预训练的Tokenizer。**这是为了确保文本的拆分方式与预训练的语料库相同，并在预训练期间使用相同的对应关系`标记-索引`（通常称为词汇表`vocab`）。

开始使用`AutoTokenizer.from_pretrained()`方法加载一个预训练的 Tokenizer，并下载预训练模型的词汇表 vocab：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

然后文本输入给 Tokenizer：

In [ ]:
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input)

Tokenizer 会返回一个包含三个重要对象的字典：

- `input_ids`：这是一个整数序列，每个整数代表词汇表中的一个特定token。**模型使用这个序列作为输入来理解和处理文本数据**。
- `token_type_ids`：这是一个与 input_ids 相同长度的序列，它表示输入序列中每个 token 的类型。这个字段主要用于**区分不同类型的输入序列**，尤其是在处理由多个文本序列组成的输入时，例如问答系统中的问题和答案，或者文本分类任务中的文本和标签。
- `attention_mask`：这个序列表示输入序列中哪些 token 是实际的文本token，哪些是填充（padding）token。在处理批量数据时，**由于不同序列的长度可能不同，通常需要使用填充token来使所有序列长度一致**。`attention_mask` 中的 `1 表示实际的文本token`，而 `0 表示填充token`，这样模型就可以忽略填充token。

通过`tokenizer.decode()`可以解码 `input_ids` 得到输入的文本：

In [ ]:
tokenizer.decode(encoded_input["input_ids"])

从解码得到的文本可知，tokenizer 在句子中添加了两个特殊 token - `[CLS]` 和 `[SEP]`（分类器和分隔符）。

**注意！并不是所有模型都需要添加特殊 token，但如果需要，tokenizer 会自动给你添加。**

如果有多个句子需要进行预处理，可以将它们作为列表传递给 tokenizer：

In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs = tokenizer(batch_sentences)
print(encoded_inputs)

### 填充